In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,341 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,157 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 http://secu

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# 1b. Create a new DataFrame with the column year.
from pyspark.sql.functions import year
df = df.withColumn("year_sold", year(df["date"]))

# 1c. Show DataFrame.
df.show()


+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|     2022|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|     2021|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|     2019|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|     2019|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|  

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

avg_prices = spark.sql("""
SELECT 
    year_sold AS year,
    AVG(price)
FROM home_sales 
WHERE bedrooms = 4
GROUP BY year 
ORDER BY year
""").collect()

#  Convert the Spark SQL query results to a list.
avg_prices_list = [row for row in avg_prices]

#  Show the list rounded to two decimal places.
for row in avg_prices_list:
    print(f"Sold in: {row[0]} Average Price: {round(row[1], 2)}")

Sold in: 2019 Average Price: 300263.7
Sold in: 2020 Average Price: 298353.78
Sold in: 2021 Average Price: 301819.44
Sold in: 2022 Average Price: 296363.88


In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

avg_prices = spark.sql("""
SELECT
    date_built,
    AVG(price)
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
""").collect()

# Convert the Spark SQL query results to a list.
avg_prices_list = [row for row in avg_prices]

# Show the list rounded to two decimal places.
for row in avg_prices_list:
    print(f"Built in the year: {row[0]} Average Price: {round(row[1], 2)}")

Built in the year: 2010 Average Price: 292859.62
Built in the year: 2011 Average Price: 291117.47
Built in the year: 2012 Average Price: 293683.19
Built in the year: 2013 Average Price: 295962.27
Built in the year: 2014 Average Price: 290852.27
Built in the year: 2015 Average Price: 288770.3
Built in the year: 2016 Average Price: 290555.07
Built in the year: 2017 Average Price: 292676.79


In [11]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_prices = spark.sql("""
SELECT
    date_built,
    AVG(price)
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
""").collect()

# Convert the Spark SQL query results to a list.
avg_prices_list = [row for row in avg_prices]

# Show the list rounded to two decimal places.
for row in avg_prices_list:
    print(f"YBuilt in year: {row[0]} Average Price: {round(row[1], 2)}")


YBuilt in year: 2010 Average Price: 285010.22
YBuilt in year: 2011 Average Price: 276553.81
YBuilt in year: 2012 Average Price: 307539.97
YBuilt in year: 2013 Average Price: 303676.79
YBuilt in year: 2014 Average Price: 298264.72
YBuilt in year: 2015 Average Price: 297609.97
YBuilt in year: 2016 Average Price: 293965.1
YBuilt in year: 2017 Average Price: 280317.58


In [13]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
SELECT
    AVG(view),
    AVG(price)
FROM home_sales
WHERE price >= 350000
""").show()

delta_nc = time.time() - start_time
print("--- %s seconds ---" % delta_nc)

avg_prices = spark.sql("""
SELECT
    AVG(view),
    AVG(price)
FROM home_sales
WHERE price >= 350000
""").collect()

#Convert the Spark SQL query results to a list.
avg_prices_list = [row for row in avg_prices]

#Show the list rounded to two decimal places.
print()
for row in avg_prices_list:
    print(f"Average View Rating: {round(row[0], 2)} Average Price: {round(row[1], 2)}")


print("--- %s seconds ---" % (time.time() - start_time))

+---------+------------------+
|avg(view)|        avg(price)|
+---------+------------------+
|   32.264|473796.26220224716|
+---------+------------------+

--- 0.2781221866607666 seconds ---

Average View Rating: 32.26 Average Price: 473796.26
--- 0.5230381488800049 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable('home_sales')

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time_c = time.time()

spark.sql("""
SELECT
    AVG(view),
    AVG(price)
FROM home_sales
WHERE price >= 350000
""").show()

delta_c = time.time() - start_time_c
print("--- %s seconds ---" % delta_c)

# Display the difference between the run times.
print()
print(f"The query runtime on the cached 'home_sales' table is: {delta_nc-delta_c} seconds faster than on the uncached table.")

+---------+------------------+
|avg(view)|        avg(price)|
+---------+------------------+
|   32.264|473796.26220224716|
+---------+------------------+

--- 1.0533759593963623 seconds ---

The query runtime on the cached 'home_sales' table is: -0.7752537727355957 seconds faster than on the uncached table.


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet('home_sales_p')

In [18]:
# 11. Read the parquet formatted data.
df_p = spark.read.parquet('home_sales_p')

In [19]:
# 12. Create a temporary table for the parquet data.
df_p.createOrReplaceTempView('home_sales_p')

In [20]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time_p = time.time()

spark.sql("""
SELECT
    AVG(view),
    AVG(price)
FROM home_sales_p
WHERE price >= 350000
""").show()

delta_p = time.time() - start_time_p
print("--- %s seconds ---" % delta_p)

# Display the difference between the run times.
print()
print(f"The query runtime on the parquet 'home_sales' table is: {delta_c-delta_p} seconds faster than on the cached table.")

+---------+------------------+
|avg(view)|        avg(price)|
+---------+------------------+
|   32.264|473796.26220224716|
+---------+------------------+

--- 0.4397263526916504 seconds ---

The query runtime on the parquet 'home_sales' table is: 0.6136496067047119 seconds faster than on the cached table.


In [21]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [22]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False